In [14]:
import torch
import kagglehub
from lightning import LightningModule, Trainer
from lightning.pytorch.callbacks import TQDMProgressBar
from torch import nn, hub, optim
from torch.utils import data
from torchvision import datasets, models, transforms
from pathlib import Path
from torchmetrics import classification
from torcheval import metrics

In [15]:
device = None
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else: device = 'cpu'

device

'cuda'

In [16]:
weights = models.Inception_V3_Weights.DEFAULT

inception_model = hub.load('pytorch/vision:v0.10.0', 'inception_v3', weights=weights)

inception_model.aux_logits=False

for param in inception_model.parameters():
    param.requires_grad = False

original_fc_in_features = inception_model.fc.in_features

preprocess_transforms = weights.transforms()
batch_size=32

preprocess_transforms

Using cache found in /home/pranay/.cache/torch/hub/pytorch_vision_v0.10.0


ImageClassification(
    crop_size=[299]
    resize_size=[342]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [17]:
data_root = kagglehub.dataset_download(
    handle='puneet6060/intel-image-classification',
)

data_root

'/home/pranay/.cache/kagglehub/datasets/puneet6060/intel-image-classification/versions/2'

In [18]:
train_path = Path('seg_train/seg_train')

train_dataset = datasets.ImageFolder(
    root=data_root/train_path,
    transform=preprocess_transforms
)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_data, val_data = data.random_split(
    dataset=train_dataset,
    lengths=[train_size, val_size]
)

image_shape = train_data[0][0].shape

train_loader = data.DataLoader(
    dataset=train_data, 
    batch_size=batch_size, 
    shuffle=True,
    num_workers=2,
    persistent_workers=True
)
val_loader = data.DataLoader(
    dataset=val_data, 
    batch_size=batch_size, 
    shuffle=False,
    num_workers=2,
    persistent_workers=True
)

num_classes = len(train_dataset.classes)
num_classes, image_shape

(6, torch.Size([3, 299, 299]))

In [19]:
test_path = Path('seg_test/seg_test')

test_data = datasets.ImageFolder(
    root=data_root/test_path,
    transform=preprocess_transforms
)
test_loader = data.DataLoader(
    dataset=test_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    persistent_workers=True
)

In [20]:
dataset = dict(
    train=train_loader,
    validate=val_loader,
    test=test_loader
)

In [21]:
inception_model.fc = nn.Sequential(
    nn.Linear(in_features=original_fc_in_features, out_features=1024),
    nn.BatchNorm1d(num_features=1024),
    nn.ReLU(),
    nn.Dropout(p=.3),
    nn.Linear(in_features=1024, out_features=256),
    nn.BatchNorm1d(num_features=256),
    nn.ReLU(),
    nn.Dropout(p=.3),
    nn.Linear(in_features=256, out_features=32),
    nn.BatchNorm1d(num_features=32),
    nn.ReLU(),
    nn.Dropout(p=.1),
    nn.Linear(in_features=32, out_features=num_classes),
)

# inception_model

In [22]:
len(dataset['train']), len(dataset['validate'])

(351, 88)

In [23]:
# define the LightningModule

class InceptionClassifier(LightningModule):
    def __init__(self, model, learning_rate=1e-3, decay_rate=.7, smoothing=.9):
        super().__init__()
        self.model = model
        self.criterion = nn.CrossEntropyLoss()
        self.train_accuracy = classification.MulticlassAccuracy(num_classes=num_classes)
        self.val_accuracy = classification.MulticlassAccuracy(num_classes=num_classes)
        self.test_accuracy = classification.MulticlassAccuracy(num_classes=num_classes)
        # self.learning_rate = learning_rate
        self.optimizer = optim.Adam(
            filter(lambda p: p.requires_grad, self.parameters()), 
            lr=learning_rate
        )
        self.scheduler = optim.lr_scheduler.ExponentialLR(
            optimizer=self.optimizer,
            gamma=decay_rate
        )
        self.smoothing = smoothing

        self.avg_loss = dict(
            train=None,
            val=None,
            test=None
        )

        self.avg_accuracy = dict(
            train=None,
            val=None,
            test=None
        )

    def update_avg_metrics(self, loss, acc, mode="train"):
        # Update moving averages
        self.avg_loss[mode] = (
            loss if self.avg_loss[mode] is None
            else self.smoothing * self.avg_loss[mode] + (1 - self.smoothing) * loss
        )
        self.avg_accuracy[mode] = (
            acc if self.avg_accuracy[mode] is None
            else self.smoothing * self.avg_accuracy[mode] + (1 - self.smoothing) * acc
        )

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch):
        inputs, targets = batch

        logits = self(inputs)

        loss = self.criterion(logits, targets)

        acc = self.train_accuracy(logits, targets)

        # self.update_avg_metrics(loss=loss, acc=acc, mode='train')

        # self.log("train_avg_loss", self.avg_loss['train'], on_step=True, on_epoch=False, prog_bar=True)
        # self.log("train_avg_acc", self.avg_accuracy['train'], on_step=True, on_epoch=False, prog_bar=True)

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True) 
        self.log("train_accuracy", self.train_accuracy, on_step=True, on_epoch=True, prog_bar=True) 

        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch

        logits = self(inputs)

        loss = self.criterion(logits, targets)

        acc = self.val_accuracy(logits, targets)

        # self.update_avg_metrics(loss=loss, acc=acc, mode='val')

        # self.log("val_avg_loss", self.avg_loss['val'], on_step=True, on_epoch=False, prog_bar=True)
        # self.log("val_avg_acc", self.avg_accuracy['val'], on_step=True, on_epoch=False, prog_bar=True)

        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True) 
        self.log("val_accuracy", self.val_accuracy, on_step=False, on_epoch=True, prog_bar=True) 

        return loss
    
    def test_step(self, batch):
        inputs, targets = batch

        logits = self(inputs)

        loss = self.criterion(logits, targets)

        acc = self.test_accuracy(logits, targets)

        # self.update_avg_metrics(loss=loss, acc=acc, mode='test')

        # self.log("test_avg_loss", self.avg_loss['test'], on_step=True, on_epoch=False, prog_bar=True)
        # self.log("test_avg_acc", self.avg_accuracy['test'], on_step=True, on_epoch=False, prog_bar=True)

        self.log("test_loss", loss, on_step=True, on_epoch=True) 
        self.log("test_accuracy", self.test_accuracy, on_step=True, on_epoch=True) 

        return loss
    
    def on_train_epoch_end(self):
        self.train_accuracy.reset()
        # self.avg_accuracy['train']=None
    
    def on_validation_epoch_end(self):
        self.val_accuracy.reset()
        # self.avg_accuracy['val']=None

    def on_test_epoch_end(self):
        self.test_accuracy.reset()
        # self.avg_accuracy['test']=None

    def configure_optimizers(self):
        return [self.optimizer], [self.scheduler]

# init the autoencoder
inception_classifier = InceptionClassifier(
    model=inception_model
)

trainer = Trainer(
    accelerator=device,
    max_epochs=5,
    enable_progress_bar=True,
    callbacks=[TQDMProgressBar(refresh_rate=5)]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [24]:
trainer.fit(
    model=inception_classifier,
    train_dataloaders=dataset['train'],
    val_dataloaders=dataset['validate'],
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | Inception3         | 27.5 M | train
1 | criterion      | CrossEntropyLoss   | 0      | train
2 | train_accuracy | MulticlassAccuracy | 0      | train
3 | val_accuracy   | MulticlassAccuracy | 0      | train
4 | test_accuracy  | MulticlassAccuracy | 0      | train
--------------------------------------------------------------
2.4 M     Trainable params
25.1 M    Non-trainable params
27.5 M    Total params
109.936   Total estimated model params size (MB)
324       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/pranay/miniconda3/envs/pyai/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MulticlassAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [25]:
trainer.test(
    model=inception_classifier,
    dataloaders=dataset['test']
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test_accuracy_epoch    │    0.9035136699676514     │
│      test_loss_epoch      │    0.27200791239738464    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 0.27200791239738464,
  'test_accuracy_epoch': 0.9035136699676514}]

In [14]:
# epochs = 5
# learning_rate=1e-3
# # decay_steps = epochs
# decay_rate = .1
# # batch_size=32

# loss_fn = nn.CrossEntropyLoss()
# optimizer = optim.Adam(
#     params = filter(lambda p: p.requires_grad, inception_model.parameters()),
#     lr=learning_rate
# )
# # lr_schedule = optim.lr_scheduler.ExponentialLR(
# #     optimizer=optimizer,
# #     gamma=decay_rate
# # )
# train_accuracy = metrics.MulticlassAccuracy()
# val_accuracy = metrics.MulticlassAccuracy()

In [15]:
# inception_model.to(device=device)
# train_accuracy.to(device=device)
# val_accuracy.to(device=device)

In [16]:
# for epoch in range(epochs):
#     inception_model.train()
#     for step, (images, labels) in enumerate(dataset['train']):
#         inception_model.zero_grad()
#         images = images.to(device=device)
#         labels = labels.to(device=device)
#         logits = inception_model(images)
        
#         loss = loss_fn(logits, labels)
#         loss.backward()
#         optimizer.step()

#         train_loss = loss.item()
#         train_accuracy.update(logits,labels)
#         t_accuracy = train_accuracy.compute()

#         if (step+1) % 10 == 0 or step==0:
#             print(f'Epoch {epoch + 1}/{epochs}: Step {step+1}/{len(dataset['train'])}: Training Loss: {train_loss}, Train Accuracy: {t_accuracy}')

#     inception_model.eval()
#     val_loss = 0.
#     v_accuracy = 0.
#     with torch.inference_mode():
#         for val_step, (val_images, val_labels) in enumerate(dataset['validate']):
#             val_images = val_images.to(device=device)
#             val_labels = val_labels.to(device=device)
#             val_logits = inception_model(val_images)
#             _val_loss = loss_fn(val_logits, val_labels)

#             val_loss = _val_loss.item()
#             val_accuracy.update(val_logits, val_labels)
#             v_accuracy = val_accuracy.compute()

#             if (val_step+1) % 10 == 0 or val_step==0:
#                 print(f'Epoch {epoch + 1}/{epochs}: Step {val_step+1}/{len(dataset['train'])}: Training Loss: {train_loss}, Train Accuracy: {t_accuracy}')
    
#     train_accuracy.reset()
#     val_accuracy.reset()
